<a href="https://colab.research.google.com/github/MIKRAMS1984/Python-Projects/blob/main/Sales_Prediction_project_Smitha_Kolan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Downloading and installing Python upgini library for data enrichement.
!pip install upgini catboost

In [7]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else " https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [8]:
# Splitting training and testing data set.
train = df[df["date"] < "2017-01-01"]
test = df[df["date"]  >= "2017-01-01"]

In [9]:
# Intializing train/test features and labels.
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

In [ ]:
# Enrich Features.
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys= {
        "date": SearchyKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])


In [23]:
# Defining catboost model.
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set = [(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

NameError: ignored

In [24]:
# adding upgini features into original data set.
enriched_train_features = enricher.transform(train_features, keep_input = True)
enriched_test_features = enricher.transform(test_features, keep_input = True)
enriched_train_features.head()

NameError: ignored

In [25]:
# training the model on original and enriched upgini featues.
model.fit(train_features, train_target)
preds = model.predict(test_features)
# smape =symmetric mean absolute error
eval_metric(test_target.values, preds, "SMAPE")

[37.65141857448004]

In [ ]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
# smape =symmetric mean absolute error
eval_metric(test_target.values, enriched_preds, "SMAPE")